In [8]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold


In [7]:

def build_neural_network(input_dim):
    model = Sequential([
        Dropout(0.08),  # Adjusted dropout rate
        Dense(512, activation='relu'),  # Increased neurons
        Dense(256, activation='relu', input_dim=input_dim),  # Changed neurons
        Dense(128, activation='relu'),  # Additional layer
        Dense(64, activation='relu'),
        Dense(1, activation='linear')  # Output layer
    ])
    optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model


# Load the dataset
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# Save row IDs for the final output
row_ids = df_test['row ID']

# Drop 'sub_area' from both datasets
df_train = df_train.drop(columns=['sub_area'])
df_test = df_test.drop(columns=['sub_area', 'row ID'])

# Identify and label encode categorical columns
categorical_columns = df_train.select_dtypes(include='object').columns
label_encoder = LabelEncoder()
for column in categorical_columns:
    df_train[column] = label_encoder.fit_transform(df_train[column])
    df_test[column] = label_encoder.transform(df_test[column])

# Encode categorical variables
X_train = pd.get_dummies(df_train, drop_first=True)
X_test = pd.get_dummies(df_test, drop_first=True)

# Separate features and target variable
features = X_train.drop(columns=['price_doc'])
target = X_train['price_doc']

# Impute missing values with the median
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(features)
X_test = imputer.transform(X_test)

# Scale features to a specific range
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Feature selection based on variance threshold
variance_threshold = 0.01  # Set your desired threshold
selector = VarianceThreshold(threshold=variance_threshold)
X_train_high_variance = selector.fit_transform(X_train_scaled)
X_test_high_variance = selector.transform(X_test_scaled)

# Perform PCA for dimensionality reduction
pca_components = 10 # Adjust based on your preference or use model evaluation
pca = PCA(n_components=pca_components)
X_train_pca = pca.fit_transform(X_train_high_variance)
X_test_pca = pca.transform(X_test_high_variance)

# Create polynomial features without interaction
poly_degree = 2  # Adjust based on your preference or use model evaluation
poly_features = PolynomialFeatures(degree=poly_degree, include_bias=False, interaction_only=False)
X_train_poly = poly_features.fit_transform(X_train_pca)
X_test_poly = poly_features.transform(X_test_pca)

# Build and train the neural network
input_dim = X_train_poly.shape[1]
model = build_neural_network(input_dim)

# Train the model for 20 epochs
model.fit(X_train_poly, target, epochs=10, batch_size=40, validation_split=0.2, verbose=1)

# Make predictions on the test set
predictions = model.predict(X_test_poly).flatten()

# Calculate RMSE on Training Set
rmse_train = sqrt(mean_squared_error(target, model.predict(X_train_poly).flatten()))



Epoch 1/10
3631/3631 [==============================] - 8s 2ms/step - loss: 220313803554816.0000 - val_loss: 178969290735616.0000
Epoch 2/10
3631/3631 [==============================] - 7s 2ms/step - loss: 191287089168384.0000 - val_loss: 177319301873664.0000
Epoch 3/10
3631/3631 [==============================] - 7s 2ms/step - loss: 189679445999616.0000 - val_loss: 173614724808704.0000
Epoch 4/10
3631/3631 [==============================] - 7s 2ms/step - loss: 190073106595840.0000 - val_loss: 174246336659456.0000
Epoch 5/10
3631/3631 [==============================] - 8s 2ms/step - loss: 186842536214528.0000 - val_loss: 172457885433856.0000
Epoch 6/10
3631/3631 [==============================] - 8s 2ms/step - loss: 184045388431360.0000 - val_loss: 174796864225280.0000
Epoch 7/10
3631/3631 [==============================] - 8s 2ms/step - loss: 181667335503872.0000 - val_loss: 176266296688640.0000
Epoch 8/10
3631/3631 [==============================] - 9s 2ms/step - loss: 18130894061568

In [12]:
X_train.shape

(181507, 270)

In [9]:
print(f"RMSE on Training Set: {rmse_train}")


RMSE on Training Set: 13336948.832428165


In [11]:
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': predictions})

# Save the DataFrame to a CSV file
result_df.to_csv('predictions_42.csv', index=False)

In [ ]:
# Create a DataFrame with predictions and 'row_ids'
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': predictions})

# Save the DataFrame to a CSV file
result_df.to_csv('predictions_neural_network.csv', index=False)

# Print/fetch details
print("Neural Network Model with 10 epochs")
print(f"RMSE on Training Set: {rmse_train}")
print("Number of Features Used:", X_train_poly.shape[1])
print("Architecture Details:")
print("Dropout Rate: 0.08")
print("Batch Size: 40")
print("Number of Epochs: 10")


In [8]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_selection import VarianceThreshold
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import time



# Load the dataset
train_df = pd.read_csv('train.csv')  # Update with your file path
test_df = pd.read_csv('test.csv')  # Update with your file path

# Save row IDs for the final output
row_ids = test_df['row ID']

# Drop 'sub_area' from both datasets
train_df = train_df.drop(columns=['sub_area'])
test_df = test_df.drop(columns=['sub_area', 'row ID'])

# Identify and label encode categorical columns
categorical_cols = train_df.select_dtypes(include='object').columns
label_encoder = LabelEncoder()

for col in categorical_cols:
    train_df[col] = label_encoder.fit_transform(train_df[col])
    test_df[col] = label_encoder.transform(test_df[col])

# Encode categorical variables
train_encoded = pd.get_dummies(train_df, drop_first=True)
test_encoded = pd.get_dummies(test_df, drop_first=True)

# Separate features and target variable
X_train = train_encoded.drop(columns=['price_doc'])
y_train = train_encoded['price_doc']

# Impute missing values with the median
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(X_train)
test_encoded = imputer.transform(test_encoded)

# Scale features to a specific range
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
test_encoded_scaled = scaler.transform(test_encoded)

# Feature selection based on variance threshold
selector = VarianceThreshold(threshold=0.01)
X_train_var = selector.fit_transform(X_train_scaled)
X_test_var = selector.transform(test_encoded_scaled)

# Perform PCA for dimensionality reduction
pca = PCA(n_components=10)
X_train_pca = pca.fit_transform(X_train_var)
X_test_pca = pca.transform(X_test_var)

# Create polynomial features without interaction
poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)
X_train_poly = poly.fit_transform(X_train_pca)
X_test_poly = poly.transform(X_test_pca)

# Build and train the neural network


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def build_neural_network(input_dim):
    model = Sequential()
    model.add(Dropout(0.08))  # Adjusted dropout rate
    model.add(Dense(256, activation='relu', input_dim=input_dim))  # Changed neurons
    model.add(Dropout(0.08))  # Adjusted dropout rate
    model.add(Dense(128, activation='relu'))  # Additional layer
    model.add(Dropout(0.08))  # Adjusted dropout rate
    model.add(Dense(1, activation='linear'))
    optimizer = Adam(learning_rate=0.001)  # Adjusted learning rate
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['RootMeanSquaredError'])
    return model

model = build_neural_network(X_train_poly.shape[1])


In [22]:

# Train the model
model.fit(X_train_poly, y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Make predictions on the test set
predictions = model.predict(X_test_poly).flatten()
# print(f"RMSE on Training Set: {rmse_train}")


Epoch 1/20
4538/4538 [==============================] - 5s 970us/step - loss: 413422512504832.0000 - root_mean_squared_error: 20332794.0000 - val_loss: 194960376725504.0000 - val_root_mean_squared_error: 13962821.0000
Epoch 2/20
4538/4538 [==============================] - 4s 847us/step - loss: 208394380115968.0000 - root_mean_squared_error: 14435871.0000 - val_loss: 188212110688256.0000 - val_root_mean_squared_error: 13719042.0000
Epoch 3/20
4538/4538 [==============================] - 4s 845us/step - loss: 206127073591296.0000 - root_mean_squared_error: 14357126.0000 - val_loss: 185252693671936.0000 - val_root_mean_squared_error: 13610757.0000
Epoch 4/20
4538/4538 [==============================] - 4s 960us/step - loss: 202336781729792.0000 - root_mean_squared_error: 14224513.0000 - val_loss: 184304697081856.0000 - val_root_mean_squared_error: 13575887.0000
Epoch 5/20
4538/4538 [==============================] - 4s 956us/step - loss: 199751110754304.0000 - root_mean_squared_error: 14

In [24]:

# Print RMSE on Training Set
print(f"RMSE on Training Set: {sqrt(mean_squared_error(y_train, model.predict(X_train_poly).flatten()))}")



5673/5673 [==============================] - 2s 388us/step
RMSE on Training Set: 26349211.29976074


In [7]:
# Create a DataFrame with predictions and 'row_ids'
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': predictions})

# Save the DataFrame to a CSV file
result_df.to_csv('predictions_43.csv', index=False)
